# Decorators

Python decorators are a powerful concept that allow you to "wrap" a function with another function. A function can be decorated many times, if desired.

* [functools](https://docs.python.org/3/library/functools.html) — Higher-order functions and operations on callable objects
* [How to use functools.wraps](https://www.blog.pythonlibrary.org/2016/02/17/python-functools-wraps/)
* [inspect — Inspect live objects](https://docs.python.org/3/library/inspect.html), [inspect.getcallargs()](https://docs.python.org/3/library/inspect.html#inspect.getcallargs)
* [PythonDecoratorLibrary](https://wiki.python.org/moin/PythonDecoratorLibrary) 👈🏻

## Functions That Return Other Functions

* When you define a function in Python, that function becomes an object.
* A _decorator_ is a function that takes another function as an argument and replace it with a new, modified function.

In [1]:
def apply(func, arg):
    return func(arg)

In [2]:
apply(len, "hello")

5

In [3]:
def add(num):
    def adder(x):
        return num + x
    return adder

In [4]:
func = add(10)

In [5]:
func(5)

15

In [6]:
add(1)(10)

11

ℹ️ Let's write a function that returns other function that validates a value. We will use the `string_required` function as a decorator in future examples.

In [7]:
def string_required(f):
    def validate(arg):
        if type(arg) != str:
            raise ValueError(f'{repr(arg)} is not a string')
        return f(arg)
    return validate

In [8]:
string_required(len)("limited len")

11

In [9]:
string_required(len)([0, 1, 2, 3])

ValueError: [0, 1, 2, 3] is not a string

### One More Example

In [10]:
_functions = {}

def register(func):
    global _functions
    _functions[func.__name__] = func
    return func

@register
def foo():
    return 'bar'

_functions['foo']

<function __main__.foo()>

## Decorators Are a Syntax Sugar

In [11]:
@string_required
def greeting(name):
    """Says Hi to a person"""
    print(f'Hi, {name}!')

In [12]:
greeting('Andrey')

Hi, Andrey!


In [13]:
# Let's check function attrubutes
greeting.__name__, greeting.__module__, greeting.__doc__

('validate', '__main__', None)

⚠️ _Please notice that our decorated function `greeting` now has the wrong attributes._

In [14]:
greeting(1)

ValueError: 1 is not a string

In [15]:
@string_required
class Greeter:
    def __init__(self, name):
        self.name = name
        
    def say(self):
        print(f'Hi, {self.name}!')

In [16]:
Greeter('Andrey').say()

Hi, Andrey!


In [17]:
Greeter(('Andrey', 'Maria', 'Mark')).say()

ValueError: ('Andrey', 'Maria', 'Mark') is not a string

---

🤔 _How we can fix wrong function attributes? Maybe by copying them one by one from decorated function?_

In [18]:
def string_required(f):
    def validate(arg):
        if type(arg) != str:
            raise ValueError(f'{repr(arg)} is not a string')
        return f(arg)
    # ---------------------------------
    validate.__module__ = f.__module__
    validate.__name__ = f.__name__
    validate.__doc__ = f.__doc__
    # ---------------------------------
    return validate

In [19]:
@string_required
def greeting(name):
    """Says Hi to a person"""
    print(f'Hi, {name}!')

In [20]:
greeting.__name__, greeting.__module__, greeting.__doc__

('greeting', '__main__', 'Says Hi to a person')

Yes, it works. But doing so is a decent amount of work and seems to be an error-prone solution.

### functools.update_wrapper()

A decorator replaces the original function with a new one built on the fly. This new function lacks many of the attributes of the original function, such as its docstring and its name. The `functools.update_wrapper()` function solves this problem. It copies the attributes from the original function that were lost to the wrapper itself.

In [21]:
import functools

In [22]:
def string_required(f):
    def validate(arg):
        if type(arg) != str:
            raise ValueError(f'{repr(arg)} is not a string')
        return f(arg)
    # -----------------------------------
    functools.update_wrapper(validate, f)
    # -----------------------------------
    return validate

### wraps() – decorator for decorators

_Another option to solve the mentioned problem is to use the `wraps` decorator._

It can get tedious to use `update_wrapper()` manually when creating decorators, so functools provides a decorator for decorators called `wraps`. With `functools.wrap()`, the decorator function that returns the `wrapper()` function takes care of copying the docstring, name function, and other information from the function `f` passed as argument.

In [23]:
def string_required(f):
    @functools.wraps(f)
    def validate(arg):
        if type(arg) != str:
            raise ValueError(f'{repr(arg)} is not a string')
        return f(arg)
    return validate

### One More Example

⚠️ This naive approach has some major drawbacks! Try to implement stacking decorators in this way.

In [24]:
def admin_required(f):
    def wrapper(*args, **kwargs):
        if kwargs.get('username') != 'admin':
            raise Exception('This user is not allowed to log in')
        return f(*args, **kwargs)
    
    return wrapper

class SiteView:
    @admin_required
    def admin_page(self, username):
        return 'Admin area'

💡 _Check the `inspect` module. It's better to use its functions rather checking whether function parameters are positional or keyword arguments._

## Tip: Optional Decorator

Let's imagine we want to configure the decorator behavior and sometimes call the original function. To do so we will define a global variable `trace_enabled` and will check its value inside the `inner` function.

In [25]:
trace_enabled = False

def trace(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner if trace_enabled else func

⚠️ _Please note that Python checks config value **during bytecode compilation**, not in runtime._

## Injecting Logic Into Class Methods

_Class decorators_ work in the same way as function decorators, but they act on classes rather than functions.

In [26]:
import uuid

def set_class_name_and_id(klass):
    klass.name = str(klass)
    klass.random_id = uuid.uuid4()
    return klass

@set_class_name_and_id
class SomeClass:
    pass

SomeClass().random_id

UUID('f956d605-3b8c-4a79-b0d6-159747cc8d04')

In [27]:
def barking(cls):
    for name in cls.__dict__:
        if name.startswith('__'):
            continue
        
        func = getattr(cls, name)

        def woofer(*args, **kwargs):
            print('Woof')
            return func(*args, **kwargs)

        setattr(cls, name, woofer)
        
    return cls

In [28]:
@barking
class Dog1:
    def shout(self):
        print("I'm a dog!")

In [29]:
d = Dog1()

In [30]:
d

In [31]:
d.shout()

Woof
I'm a dog!


Class decorators are useful for wrapping a function that's storing a state.

In [32]:
class CountCalls:
    def __init__(self, f):
        self.f = f
        self.called = 0
        
    def __call__(self, *args, **kwargs):
        self.called += 1
        return self.f(*args, **kwargs)
    
@CountCalls
def greeting():
    print("Hi")
    
greeting.called

0

In [33]:
greeting()
greeting()
greeting()

Hi
Hi
Hi


In [34]:
greeting.called

3

## Decorators Can Return Something That Is Not a Function

The decorator has the ability to swallow the function, or return something that is not a function, if it wanted.

In [35]:
def stupid(cls):
    class Null:
        pass
    return Null

In [36]:
@stupid
class Something:
    def method1(self):
        pass
    def method2(self):
        pass

In [37]:
Something

__main__.stupid.<locals>.Null

In [38]:
Something.method1()

AttributeError: type object 'Null' has no attribute 'method1'

## Decorators With Arguments

In [39]:
def trace(handle):
    def decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs, file=handle)
            return func(*args, **kwargs)
        return inner
    return decorator

### Decorator For Decorators

In [40]:
def with_arguments(deco):
    @functools.wraps(deco)
    def wrapper(*dargs, **dkwargs):
        def decorator(func):
            result = deco(func, *dargs, **dkwargs)
            functools.update_wrapper(result, func)
            return result
        return decorator
    return wrapper

In [41]:
@with_arguments
def trace(func, handle):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs, file=handle)
        return func(*args, **kwargs)
    return inner

In [42]:
import sys

@trace(sys.stderr)
def identity(x):
    return x

In [43]:
identity(1)

identity (1,) {}


1

In [44]:
@trace
def identity(x):
    return x

In [45]:
# Doesn't work without an argument
identity(1)

<function __main__.trace.<locals>.inner>

###  Better / Simpler Option

In [46]:
def trace(func=None, *, handle=sys.stdout):
    if func is None:
        return lambda func: trace(func, handle=handle)
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs, file=handle)
        return func(*args, **kwargs)
    return inner

ℹ️ _The decorator argument is optional here._

In [47]:
@trace
def identity(x):
    return x

In [48]:
identity('Andrey')

identity ('Andrey',) {}


'Andrey'

In [49]:
@trace(sys.stderr)
def identity(x):
    return x

AttributeError: 'OutStream' object has no attribute '__name__'

⚠️ _A keyword argument is required._

In [50]:
@trace(handle=sys.stderr)
def identity(x):
    return x

In [51]:
identity(1)

identity (1,) {}


1

## More Examples

### timethis

In [55]:
import time

In [56]:
def timethis(func=None, *, n_iter=100):
    if func is None:
        return lambda func: timethis(func, n_iter=n_iter)
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=' ... ')
        acc = float('inf')
        for i in range(n_iter):
            tick = time.perf_counter()
            result = func(*args, **kwargs)
            acc = min(acc, time.perf_counter() - tick)
        print(acc)
        return result
    return inner

In [57]:
result = timethis(sum)(range(10**6))

sum ... 0.018446182999980465


### profiled

In [58]:
def profiled(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)
    
    inner.ncalls = 0
    return inner

In [59]:
@profiled
def identity(x):
    return x

In [60]:
identity(42)

42

In [61]:
identity.ncalls

1

### once

In [62]:
def once(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        if not inner.called:
            func(*args, **kwargs)
            inner.called = True
    inner.called = False
    return inner

In [63]:
@once
def initialize_settings():
    print('Settings has been initialized.')

In [64]:
initialize_settings()

Settings has been initialized.


In [65]:
initialize_settings()

🤔 Question: How we can modify this decorator to support functions that return something not equal to `None`.

### memoized

[Memoization - Wikipedia](https://en.wikipedia.org/wiki/Memoization)

> In computing, memoization or memoisation is an optimization technique used primarily to speed up computer programs by storing the results of expensive function calls and returning the cached result when the same inputs occur again.

In [66]:
def memoized(func):
    cache = {}
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args, kwargs  # yes, you're right, this wouldn't work
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return inner

In [68]:
@memoized
def ackerman(m, n):
    if not m:
        return n + 1
    elif not n:
        return ackerman(m - 1, 1)
    else:
        return ackerman(m - 1, ackerman(m, n - 1))

In [69]:
ackerman(3, 4)

TypeError: unhashable type: 'dict'

We can fix this problem by finding a better option for building a hashable key. For example, we can serialize function arguments into a string or by using pickle.

In [77]:
def memoized(func):
    cache = {}
    
    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args + tuple(sorted(kwargs.items()))  # works in most of the cases
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return inner

In [78]:
@memoized
def ackerman(m, n):
    if not m:
        return n + 1
    elif not n:
        return ackerman(m - 1, 1)
    else:
        return ackerman(m - 1, ackerman(m, n - 1))

In [79]:
ackerman(3, 4)

125

### deprecated

In [85]:
import warnings


def deprecated(func):
    code = func.__code__
    warnings.warn_explicit(
        func.__name__ + ' is deprecated.',
        category=DeprecationWarning,
        filename=code.co_filename,
        lineno=code.co_firstlineno + 1
    )
    return func

In [86]:
@deprecated
def identity(x):
    return x

### Contract Programming

[Contract Programming – D Programming Language](https://dlang.org/spec/contracts.html)

> Contracts are a breakthrough technique to reduce the programming effort for large projects. Contracts are the concept of preconditions, postconditions, errors, and invariants.

https://pypi.python.org/pypi/contracts

We already saw the `string_required` decorator above. Let's write another decorator that will help to validate data by calling condition function.

In [88]:
def post(cond, message):
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            result = func(*args, **kwargs)
            assert cond(result), message
            return result
        return inner
    return wrapper

In [91]:
import math


@post(lambda x: not math.isnan(x), 'not a number')
def something_useful():
    return float('nan')

In [92]:
something_useful()

AssertionError: not a number

## Multiple Decorators For The Same Function

- Python allows applying multiple decorators for the same function
- Order of decorators does matter

In [93]:
def square(func):
    return lambda x: func(x * x)


def add42(func):
    return lambda x: func(x + 42)

In [94]:
@square
@add42
def identity(x):
    return x

In [95]:
identity(2)

46

In [96]:
@add42
@square
def identity(x):
    return x

In [97]:
identity(2)

1936

## functools

### lru_cache

Instead of writing memoization by hand, we can use the LRU cache decorator.

[Documentation](https://docs.python.org/3/library/functools.html#functools.lru_cache)

```python
@functools.lru_cache(maxsize=64)
@functools.lru_cache(maxsize=None) # not so good idea, why? it can cause memory leak
```

### partial

[Documentation](https://docs.python.org/3/library/functools.html#functools.partial)

> Return a new partial object which when called will behave like func called with the positional arguments args and keyword arguments keywords.

In [98]:
f = functools.partial(sorted, key=lambda p: p[1])

In [99]:
f([('a', 4), ('b', 2)])

[('b', 2), ('a', 4)]

In [101]:
g = functools.partial(sorted, [2, 3, 1, 4])

In [102]:
g()

[1, 2, 3, 4]

### Generic Functions

[PEP-0443](https://www.python.org/dev/peps/pep-0443/)

`len`, `str`, `hash`, `sum` are examples of a generic function.

In [103]:
len([1, 2, 3, 4])

4

In [104]:
len({1, 2, 3, 4})

4

In [105]:
len(range(4))

4

#### singledispatch

In [106]:
@functools.singledispatch
def pack(obj):
    type_name = type(obj).__name__
    assert False, 'Unsupported type: ' + type_name

In [107]:
@pack.register(int)
def _(obj):
    return b'I' + hex(obj).encode('ascii')

In [108]:
@pack.register(list)
def _(obj):
    return b'L' + b','.join(map(pack, obj))

In [109]:
pack([1, 2, 3])

b'LI0x1,I0x2,I0x3'

In [110]:
pack(42.)

AssertionError: Unsupported type: float

### reduce

[Documentation](https://docs.python.org/3/library/functools.html#functools.reduce)

In [111]:
functools.reduce(lambda acc, x: acc * x, [1, 2, 3, 4])

24